In [1]:
import h5py
import numpy as np
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

In [4]:
input_file = '/mnt/f/hprc/hprc-v1.1-mc-chm13_segments.gfa'
output_file = '/mnt/f/hprc/segments_b.hdf5'

In [7]:
nucleotide_to_index = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
dt = h5py.vlen_dtype(np.uint8)
max_seq_count = 100000

In [8]:
seq_added = 0
with h5py.File(output_file, 'w') as hdf5_file:
    sequences_dataset = hdf5_file.create_dataset('sequences', shape=(0,), maxshape=(None,), dtype=dt)

    with open(input_file, 'r') as f:
        for line in tqdm(f, desc="Processing sequences"):
            seq = line.strip()
            if not seq:
                continue  # Skip empty lines

            # Encode the sequence
            encoded_seq = np.array([nucleotide_to_index.get(nuc, 4) for nuc in seq], dtype=np.uint8)
            # '4' can represent 'N' or unknown nucleotides

            # Resize the dataset and store the sequence
            current_size = sequences_dataset.shape[0]
            sequences_dataset.resize((current_size + 1,))
            sequences_dataset[current_size] = encoded_seq

            seq_added += 1
            if seq_added == max_seq_count:
                break

Processing sequences: 99999it [00:05, 16731.69it/s]
